# Check Product Propensity Customer Table
* StelllarAlgo Data Science
* Ryan Kazmerik
* Oct 21, 2022

In [38]:
import boto3
import pandas as pd

from shared_utilities import helpers

In [39]:
session = helpers.get_aws_session("Stellaralgo-DataScienceAdmin")

Authorized as AROASQ4JELIXYLYV6P4UV:gdonst@stellaralgo.com


In [40]:
CLUSTER = "prod-app"
DATABASE = "stlrnfl49ers"
LKUPCLIENTID = 95

CNXN = helpers.get_redshift_connection(CLUSTER, DATABASE)

Authorized as AROASQ4JELIXYLYV6P4UV:gdonst@stellaralgo.com


In [41]:
sql= f"""
    SELECT * FROM {DATABASE}.dw.customerretentionscores
    WHERE updatedatetime = (
        SELECT MAX(updatedatetime) FROM {DATABASE}.dw.customerretentionscores
    )
    AND lkupclientid = {LKUPCLIENTID}
"""

try:
    cursor = CNXN.cursor()
    cursor.execute(sql)
    data = cursor.fetchall()

    cols = [row[0] for row in cursor.description]
    df_scores = pd.DataFrame(data=data, columns=cols)

    print(f"DATABASE: {DATABASE}")
    print(f"ROWS RETRIEVED: {df_scores.shape[0]}")
    print(f"MAX SCORE DATE: {df_scores['updatedatetime'].max()}")

except Exception as e:
    print(f"ERROR:, {e}")
    cursor.execute("ROLLBACK")

finally:
    cursor.close()


DATABASE: stlrnfl49ers
ROWS RETRIEVED: 10176
MAX SCORE DATE: 2022-11-04 18:35:55


In [42]:
df_scores.groupby(["product","product_current"]).agg({
    "product": "count",
})

KeyError: 'product_current'

### Done